## 5.3. Formatting Meteorological Datasets for UMEP Processing

First check downloaded .csv is in the right format

In [ ]:
import pandas as pd
from datetime import datetime
import os

# ---- CONFIGURATION ----
INPUT_CSV = r"\\coafs8.ad.gatech.edu\REMAP\NO_HEAT_RB\City_Atlanta\Raw\Met_Data\file20240601-20240831.csv" # Your CSV file path

df = pd.read_csv(INPUT_CSV, skiprows=11)
df.head()

,datetime_lst,datetime,t2m,stl3,stl4,sp,ssrd,strd,tcc,tp,...,sd,wdir10,ws10,ws100,r2m,alpha_sol,azimuth_sol,CF100,dhi,dni
0,2024-06-01 00:00:00,2024-06-01 05:00:00,15.3,17.5,12.4,102019,0,305,0.0,0.0,...,0,256,2.0,6.1,0.79,0.0,-3.00,0.14,0,0
1,2024-06-01 01:00:00,2024-06-01 06:00:00,14.5,17.5,12.4,101982,0,303,0.0,0.0,...,0,266,1.8,5.4,0.83,0.0,3.04,0.09,0,0
2,2024-06-01 02:00:00,2024-06-01 07:00:00,14.1,17.5,12.4,101952,0,299,0.0,0.0,...,0,271,1.9,4.9,0.84,0.0,2.80,0.06,0,0
3,2024-06-01 03:00:00,2024-06-01 08:00:00,13.3,17.5,12.4,101949,0,297,0.0,0.0,...,0,276,1.9,4.7,0.86,0.0,2.57,0.05,0,0
4,2024-06-01 04:00:00,2024-06-01 09:00:00,13.2,17.5,12.4,101966,0,296,0.0,0.0,...,0,271,2.1,5.5,0.87,0.0,2.34,0.10,0,0


Now, you can format date and time into the right format.

In [ ]:
import pandas as pd
from datetime import datetime
import os

### --- Configuration (User MUST update these paths) --- ###
INPUT_CSV = r"\\coafs8.ad.gatech.edu\REMAP\NO_HEAT_RB\City_Atlanta\Raw\Met_Data\file20240601-20240831.csv" # Your CSV file path
datetime_format = "%Y-%m-%d %H:%M:%S"     # format used in input csv datetime column

# ENTER SELECTED TIMES HERE. It should be in LOCAL Time.
selected_times = [
    "2024-06-01 13:00:00",
    "2024-06-01 14:00:00",
    "2024-06-01 15:00:00"
]

OUTPUT_TXT = r"\\coafs8.ad.gatech.edu\REMAP\NO_HEAT_RB\City_Atlanta\Raw\Met_Data\formatted.txt"  # Output txt path
### ---------------------------------------------------- ###

In [ ]:
import pandas as pd
from datetime import datetime
import os

# ---- CONFIGURATION ----
INPUT_CSV = r"\\coafs8.ad.gatech.edu\REMAP\NO_HEAT_RB\City_Atlanta\Raw\Met_Data\file20240601-20240831.csv" # Your CSV file path
OUTPUT_TXT = r"\\coafs8.ad.gatech.edu\REMAP\NO_HEAT_RB\City_Atlanta\Raw\Met_Data\formatted.txt"  # Output txt path
datetime_format = "%Y-%m-%d %H:%M:%S"     # format used in input csv datetime column

# column mappings
valid_columns = {
    'datetime_utc': 'datetime',   # Local DATETIME
    'datetime_lst': 'datetime_lst',
    'Tair': 't2m',                 # 2m temperature
    'pres': 'sp',                 # surface pressure
    'kdown': 'ssrd',              # surface solar radiation downwards
    'Idown': 'strd',              # surface long radiation downwards
    'fcld': 'tcc',                 # cloud cover
    'rain': 'tp',                 # total precipitation
    'wdir': 'wdir10',               # wind direction
    'wspeed': 'ws10',                  # 10 m wind speed
    'RH': 'r2m',                   # 2 m relative humidity
    'Kdiff' : 'dhi',
    'Kdir' : 'dni',
    
}

df = pd.read_csv(INPUT_CSV, skiprows=11)
df_filtered = df[df[valid_columns['datetime_lst']].isin(selected_times)] # EXTRACTING UTC FORMAT
print(f"Selected UTC Time:\n {df_filtered['datetime']} \n Local Time: \n {df_filtered['datetime_lst']}")

def extract_time_parts(dt_str):
    dt = datetime.strptime(dt_str, datetime_format)
    return dt.year, dt.timetuple().tm_yday, dt.hour, dt.minute

df_format = df_filtered.copy()
df_format[['iy', 'id', 'it', 'imin']] = df_filtered[valid_columns['datetime_utc']].apply(lambda x: pd.Series(extract_time_parts(x)))
df_format['pres_kPa'] = df_format[valid_columns['pres']] / 1000.0 # pa -> kpa
df_format['r2m_pct'] = df_format[valid_columns['RH']] * 100.0 # humidity -> %
print(df_format)

# write to output file
with open(OUTPUT_TXT, 'w') as f:
    # header row
    f.write("%iy  id  it imin   Q*      QH      QE      Qs      Qf    Wind    RH     Td     press   rain    Kdn    snow    ldown   fcld    wuh     xsmd    lai_hr  Kdiff   Kdir    Wd\n")
    
    for _, row in df_format.iterrows():
        line = [
            int(row['iy']), int(row['id']), int(row['it']), int(row['imin']),
            *[-999.00]*5,                          # Q*, QH, QE, Qs, Qf
            float(row[valid_columns['wspeed']]),        # Wind
            float(row['r2m_pct']),       # RH
            float(row[valid_columns['Tair']]),     # Td (temp)
            float(row['pres_kPa']),                # press
            float(row[valid_columns['rain']]),     # rain
            float(row[valid_columns['kdown']]),    # Kdn (Incoming shortwavie radiation)
            *[-999.00]*1,                           # snow up )snow_
            float(row[valid_columns['Idown']]),     #Idown (thermal radiation)
            float(row[valid_columns['fcld']]),      # fcld (cloud fraction)
            *[-999.0]*3,                            # wuh(external water use), xsmd(soil moisture), lai_hr(leaf area imndex)
            float(row[valid_columns['Kdiff']]),    # Kdiff (Diffuse shortawave radation [W m-2])
            float(row[valid_columns['Kdir']]),     # Kdir (Direct shortwave radiation [W m-2])
            float(row[valid_columns['wdir']]),  #Wind direction (°)
        ]
        f.write(" ".join(f"{x:.2f}" if isinstance(x, float) else str(x) for x in line) + "\n")

print("\n Finished. Remember that the format time is in 'UTC' Time Format. If you are using EST, you have to -5 to know local time")

Selected UTC Time:
 13    2024-06-01 18:00:00
14    2024-06-01 19:00:00
15    2024-06-01 20:00:00
Name: datetime, dtype: object 
 Local Time: 
 13    2024-06-01 13:00:00
14    2024-06-01 14:00:00
15    2024-06-01 15:00:00
Name: datetime_lst, dtype: object
           datetime_lst             datetime   t2m  stl3  stl4      sp  ssrd  \
13  2024-06-01 13:00:00  2024-06-01 18:00:00  25.8  17.3  12.4  101843   978   
14  2024-06-01 14:00:00  2024-06-01 19:00:00  26.1  17.3  12.4  101801   940   
15  2024-06-01 15:00:00  2024-06-01 20:00:00  26.4  17.3  12.4  101717   831   

    strd   tcc   tp  ...  azimuth_sol  CF100  dhi  dni    iy   id  it  imin  \
13   332  0.08  0.0  ...        -0.34    0.0  154  862  2024  153  18     0   
14   330  0.00  0.0  ...        -0.94    0.0  114  906  2024  153  19     0   
15   333  0.04  0.0  ...        -1.27    0.0  107  882  2024  153  20     0   

    pres_kPa  r2m_pct  
13   101.843     34.0  
14   101.801     35.0  
15   101.717     35.0  

[3 rows x

In [ ]:
df_format['RJ']

13    0.34
14    0.35
15    0.35
Name: r2m, dtype: float64